<a href="https://colab.research.google.com/github/mewon1/Trading-at-the-close-Forecasting/blob/main/ML_final1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Import Libraries ##  


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor


##GET DATA##  

In [ ]:
df_train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/train.csv")
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test.csv')
df_reveal= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/revealed_targets.csv')
df_train

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
0,0,0,0,3180602.69,1,0.999812,13380276.64,NaN,NaN,0.999812,60651.50,1.000026,8493.03,1.000000,-3.029704,0,0_0_0
1,1,0,0,166603.91,-1,0.999896,1642214.25,NaN,NaN,0.999896,3233.04,1.000660,20605.09,1.000000,-5.519986,0,0_0_1
2,2,0,0,302879.87,-1,0.999561,1819368.03,NaN,NaN,0.999403,37956.00,1.000298,18995.00,1.000000,-8.389950,0,0_0_2
3,3,0,0,11917682.27,-1,1.000171,18389745.62,NaN,NaN,0.999999,2324.90,1.000214,479032.40,1.000000,-4.010200,0,0_0_3
4,4,0,0,447549.96,-1,0.999532,17860614.95,NaN,NaN,0.999394,16485.54,1.000016,434.10,1.000000,-7.349849,0,0_0_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5237975,195,480,540,2440722.89,-1,1.000317,28280361.74,0.999734,0.999734,1.000317,32257.04,1.000434,319862.40,1.000328,2.310276,26454,480_540_195
5237976,196,480,540,349510.47,-1,1.000643,9187699.11,1.000129,1.000386,1.000643,205108.40,1.000900,93393.07,1.000819,-8.220077,26454,480_540_196
5237977,197,480,540,0.00,0,0.995789,12725436.10,0.995789,0.995789,0.995789,16790.66,0.995883,180038.32,0.995797,1.169443,26454,480_540_197
5237978,198,480,540,1000898.84,1,0.999210,94773271.05,0.999210,0.999210,0.998970,125631.72,0.999210,669893.00,0.999008,-1.540184,26454,480_540_198


In [ ]:
df_reveal

,stock_id,date_id,seconds_in_bucket,time_id,revealed_target,revealed_date_id,revealed_time_id
0,0.0,478,0,26290,-2.310276,477.0,26235.0
1,1.0,478,0,26290,-12.850165,477.0,26235.0
2,2.0,478,0,26290,-0.439882,477.0,26235.0
3,3.0,478,0,26290,7.259846,477.0,26235.0
4,4.0,478,0,26290,4.780292,477.0,26235.0
...,...,...,...,...,...,...,...
33157,NaN,480,500,26450,NaN,NaN,NaN
33158,NaN,480,510,26451,NaN,NaN,NaN
33159,NaN,480,520,26452,NaN,NaN,NaN
33160,NaN,480,530,26453,NaN,NaN,NaN


In [ ]:
df_train = df_train.fillna(df_train.median())
                                             ## I ma not sure if we are going to drop the empty rows. we may fill it with the mean or use sum.
df_train

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
0,0,0,0,3180602.69,1,0.999812,13380276.64,0.999883,0.999889,0.999812,60651.50,1.000026,8493.03,1.000000,-3.029704,0,0_0_0
1,1,0,0,166603.91,-1,0.999896,1642214.25,0.999883,0.999889,0.999896,3233.04,1.000660,20605.09,1.000000,-5.519986,0,0_0_1
2,2,0,0,302879.87,-1,0.999561,1819368.03,0.999883,0.999889,0.999403,37956.00,1.000298,18995.00,1.000000,-8.389950,0,0_0_2
3,3,0,0,11917682.27,-1,1.000171,18389745.62,0.999883,0.999889,0.999999,2324.90,1.000214,479032.40,1.000000,-4.010200,0,0_0_3
4,4,0,0,447549.96,-1,0.999532,17860614.95,0.999883,0.999889,0.999394,16485.54,1.000016,434.10,1.000000,-7.349849,0,0_0_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5237975,195,480,540,2440722.89,-1,1.000317,28280361.74,0.999734,0.999734,1.000317,32257.04,1.000434,319862.40,1.000328,2.310276,26454,480_540_195
5237976,196,480,540,349510.47,-1,1.000643,9187699.11,1.000129,1.000386,1.000643,205108.40,1.000900,93393.07,1.000819,-8.220077,26454,480_540_196
5237977,197,480,540,0.00,0,0.995789,12725436.10,0.995789,0.995789,0.995789,16790.66,0.995883,180038.32,0.995797,1.169443,26454,480_540_197
5237978,198,480,540,1000898.84,1,0.999210,94773271.05,0.999210,0.999210,0.998970,125631.72,0.999210,669893.00,0.999008,-1.540184,26454,480_540_198


In [ ]:
df_train.columns
col_bfor_droped = set(df_train.columns)
col_bfor_droped

{'ask_price',
 'ask_size',
 'bid_price',
 'bid_size',
 'date_id',
 'far_price',
 'imbalance_buy_sell_flag',
 'imbalance_size',
 'matched_size',
 'near_price',
 'reference_price',
 'row_id',
 'seconds_in_bucket',
 'stock_id',
 'target',
 'time_id',
 'wap'}

In [ ]:
df_test1 = df_test.fillna(df_test.median())

df_test1

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,time_id,row_id,currently_scored
0,0,478,0,3753451.43,-1,0.999875,11548975.43,0.999833,0.999873,0.999875,22940.00,1.000050,9177.60,1.000000,26290,478_0_0,False
1,1,478,0,985977.11,-1,1.000245,3850033.97,0.999833,0.999873,0.999940,1967.90,1.000601,19692.00,1.000000,26290,478_0_1,False
2,2,478,0,599128.74,1,1.000584,4359198.25,0.999833,0.999873,0.999918,4488.22,1.000636,34955.12,1.000000,26290,478_0_2,False
3,3,478,0,2872317.54,-1,0.999802,27129551.64,0.999833,0.999873,0.999705,16082.04,1.000189,10314.00,1.000000,26290,478_0_3,False
4,4,478,0,740059.14,-1,0.999886,8880890.78,0.999833,0.999873,0.999720,19012.35,1.000107,7245.60,1.000000,26290,478_0_4,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32995,195,480,540,2440722.89,-1,1.000317,28280361.74,0.999734,0.999734,1.000317,32257.04,1.000434,319862.40,1.000328,26454,480_540_195,False
32996,196,480,540,349510.47,-1,1.000643,9187699.11,1.000129,1.000386,1.000643,205108.40,1.000900,93393.07,1.000819,26454,480_540_196,False
32997,197,480,540,0.00,0,0.995789,12725436.10,0.995789,0.995789,0.995789,16790.66,0.995883,180038.32,0.995797,26454,480_540_197,False
32998,198,480,540,1000898.84,1,0.999210,94773271.05,0.999210,0.999210,0.998970,125631.72,0.999210,669893.00,0.999008,26454,480_540_198,False


In [ ]:
test_col = df_test1.columns
test_col

Index(['stock_id', 'date_id', 'seconds_in_bucket', 'imbalance_size',
       'imbalance_buy_sell_flag', 'reference_price', 'matched_size',
       'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price',
       'ask_size', 'wap', 'time_id', 'row_id', 'currently_scored'],
      dtype='object')

In [ ]:
x = df_train.drop('target', axis=1)
y = df_train['target']
x
#y

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,time_id,row_id
0,0,0,0,3180602.69,1,0.999812,13380276.64,0.999883,0.999889,0.999812,60651.50,1.000026,8493.03,1.000000,0,0_0_0
1,1,0,0,166603.91,-1,0.999896,1642214.25,0.999883,0.999889,0.999896,3233.04,1.000660,20605.09,1.000000,0,0_0_1
2,2,0,0,302879.87,-1,0.999561,1819368.03,0.999883,0.999889,0.999403,37956.00,1.000298,18995.00,1.000000,0,0_0_2
3,3,0,0,11917682.27,-1,1.000171,18389745.62,0.999883,0.999889,0.999999,2324.90,1.000214,479032.40,1.000000,0,0_0_3
4,4,0,0,447549.96,-1,0.999532,17860614.95,0.999883,0.999889,0.999394,16485.54,1.000016,434.10,1.000000,0,0_0_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5237975,195,480,540,2440722.89,-1,1.000317,28280361.74,0.999734,0.999734,1.000317,32257.04,1.000434,319862.40,1.000328,26454,480_540_195
5237976,196,480,540,349510.47,-1,1.000643,9187699.11,1.000129,1.000386,1.000643,205108.40,1.000900,93393.07,1.000819,26454,480_540_196
5237977,197,480,540,0.00,0,0.995789,12725436.10,0.995789,0.995789,0.995789,16790.66,0.995883,180038.32,0.995797,26454,480_540_197
5237978,198,480,540,1000898.84,1,0.999210,94773271.05,0.999210,0.999210,0.998970,125631.72,0.999210,669893.00,0.999008,26454,480_540_198


In [ ]:
df_test2 = df_test1.drop('currently_scored', axis=1)
df_test2

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,time_id,row_id
0,0,478,0,3753451.43,-1,0.999875,11548975.43,0.999833,0.999873,0.999875,22940.00,1.000050,9177.60,1.000000,26290,478_0_0
1,1,478,0,985977.11,-1,1.000245,3850033.97,0.999833,0.999873,0.999940,1967.90,1.000601,19692.00,1.000000,26290,478_0_1
2,2,478,0,599128.74,1,1.000584,4359198.25,0.999833,0.999873,0.999918,4488.22,1.000636,34955.12,1.000000,26290,478_0_2
3,3,478,0,2872317.54,-1,0.999802,27129551.64,0.999833,0.999873,0.999705,16082.04,1.000189,10314.00,1.000000,26290,478_0_3
4,4,478,0,740059.14,-1,0.999886,8880890.78,0.999833,0.999873,0.999720,19012.35,1.000107,7245.60,1.000000,26290,478_0_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32995,195,480,540,2440722.89,-1,1.000317,28280361.74,0.999734,0.999734,1.000317,32257.04,1.000434,319862.40,1.000328,26454,480_540_195
32996,196,480,540,349510.47,-1,1.000643,9187699.11,1.000129,1.000386,1.000643,205108.40,1.000900,93393.07,1.000819,26454,480_540_196
32997,197,480,540,0.00,0,0.995789,12725436.10,0.995789,0.995789,0.995789,16790.66,0.995883,180038.32,0.995797,26454,480_540_197
32998,198,480,540,1000898.84,1,0.999210,94773271.05,0.999210,0.999210,0.998970,125631.72,0.999210,669893.00,0.999008,26454,480_540_198


In [ ]:
test_col = df_test2.columns
test_col

Index(['stock_id', 'date_id', 'seconds_in_bucket', 'imbalance_size',
       'imbalance_buy_sell_flag', 'reference_price', 'matched_size',
       'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price',
       'ask_size', 'wap', 'time_id', 'row_id'],
      dtype='object')

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, shuffle=True)

xtrain


,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,time_id,row_id
2399819,3,222,340,466818.96,-1,0.998354,1.028296e+08,0.997746,0.998022,0.998299,11366.46,0.998520,2706.90,0.998478,12244,222_340_3
272362,138,25,440,0.00,0,1.001176,4.899948e+05,1.001176,1.001176,1.000084,23296.68,1.001176,3520.32,1.001033,1419,25_440_138
2138213,82,198,390,289917.88,1,1.006328,4.527470e+06,1.016208,1.011974,1.004917,150075.36,1.006328,64305.47,1.005905,10929,198_390_82
3579898,118,330,140,9215527.60,1,1.000968,1.054884e+07,0.999883,0.999889,0.999222,96527.72,1.001146,957.56,1.001127,18164,330_140_118
5185167,187,476,100,4015499.89,-1,0.998822,1.896997e+07,0.999883,0.999889,0.998822,13138.00,0.998974,15768.00,0.998891,26190,476_100_187
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4175911,131,384,240,6310465.26,1,1.000421,9.416869e+06,0.999883,0.999889,1.000171,21705.60,1.000421,16842.02,1.000312,21144,384_240_131
4873004,24,447,450,263594.01,1,0.999218,1.000446e+07,1.002477,0.999683,0.999101,65238.40,0.999218,51939.25,0.999166,24630,447_450_24
1129232,100,105,390,0.00,0,0.997795,3.590188e+06,0.997795,0.997795,0.997214,32623.00,0.997795,2989.32,0.997746,5814,105_390_100
999072,159,93,380,160964.93,1,1.000164,3.136874e+07,1.000817,1.000817,1.000000,14210.00,1.000164,2021.58,1.000143,5153,93_380_159


##Time series##

##Standardization##

In [ ]:
scaler = StandardScaler()
xtrain = scaler.fit_transform(xtrain)
xtest = scaler.fit_transform(xtest)
df_test3 = scaler.fit_transform(df_test2)
x = scaler.fit_transform(x)

#xtrain
df_test3


array([[-1.7234121 , -1.22474487, -1.70084013, ..., -0.08372725,
        -1.72158515, -1.00238721],
       [-1.70609137, -1.22474487, -1.70084013, ..., -0.08372725,
        -1.72158515, -1.0023872 ],
       [-1.68877065, -1.22474487, -1.70084013, ..., -0.08372725,
        -1.72158515, -1.0023872 ],
       ...,
       [ 1.68877065,  1.22474487,  1.70084013, ..., -2.37063002,
         1.72158515,  1.20484954],
       [ 1.70609137,  1.22474487,  1.70084013, ..., -0.62348636,
         1.72158515,  1.20484955],
       [ 1.7234121 ,  1.22474487,  1.70084013, ...,  1.15358346,
         1.72158515,  1.20484955]])

In [ ]:
column_names = test_col
df_test4 = pd.DataFrame(df_test3, columns=column_names)

df_test4

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,time_id,row_id
0,-1.723412,-1.224745,-1.70084,-0.156134,-0.928174,-0.145821,-0.232673,0.044787,0.082613,-0.020897,-0.387016,-0.193738,-0.405179,-0.083727,-1.721585,-1.002387
1,-1.706091,-1.224745,-1.70084,-0.305987,-0.928174,0.049912,-0.278093,0.044787,0.082613,0.012762,-0.588144,0.105676,-0.330875,-0.083727,-1.721585,-1.002387
2,-1.688771,-1.224745,-1.70084,-0.326934,1.250250,0.229245,-0.275089,0.044787,0.082613,0.001370,-0.563974,0.124695,-0.223012,-0.083727,-1.721585,-1.002387
3,-1.671450,-1.224745,-1.70084,-0.203846,-0.928174,-0.184438,-0.140757,0.044787,0.082613,-0.108929,-0.452785,-0.118205,-0.397148,-0.083727,-1.721585,-1.002387
4,-1.654129,-1.224745,-1.70084,-0.319303,-0.928174,-0.140002,-0.248414,0.044787,0.082613,-0.101161,-0.424683,-0.162764,-0.418832,-0.083727,-1.721585,-1.002387
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32995,1.654129,1.224745,1.70084,-0.227216,-0.928174,0.088000,-0.133968,0.038607,0.068483,0.207986,-0.297662,0.014928,1.790393,0.094741,1.721585,1.204850
32996,1.671450,1.224745,1.70084,-0.340450,-0.928174,0.260457,-0.246604,0.063265,0.134761,0.376799,1.360034,0.268153,0.189962,0.361900,1.721585,1.204850
32997,1.688771,1.224745,1.70084,-0.359375,0.161038,-2.307345,-0.225733,-0.207660,-0.332541,-2.136767,-0.445990,-2.458091,0.802273,-2.370630,1.721585,1.204850
32998,1.706091,1.224745,1.70084,-0.305179,1.250250,-0.497611,0.258302,0.005896,0.015216,-0.489537,0.597829,-0.650195,4.264017,-0.623486,1.721585,1.204850


In [ ]:
xtrain.shape

(4190384, 16)

In [ ]:
df_test4.shape

(33000, 16)

In [ ]:
ytrain.shape

(4190384,)

## PCA ###

In [ ]:
cov_matrix = np.cov(xtrain.T)

# Step 1: Calculate the eigenvalues and eigenvectors
eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)

# Step 2: Sort the eigenvectors by descending eigenvalues
# Get the indices of the sorted eigenvalues
sorted_indices = np.argsort(eigenvalues)[::-1]

# Sort the eigenvectors according to the same indices
sorted_eigenvectors = eigenvectors[:, sorted_indices]

# Step 3: Select the top 2 eigenvectors
projection_matrix = sorted_eigenvectors[:, :2]

# Step 4: Transform the original dataset
pca_X = x.dot(projection_matrix)

# Preparing for visualization
pca_X


array([[-0.11539817, -2.99483375],
       [-0.01021053, -3.0199673 ],
       [ 0.21552596, -2.99629799],
       ...,
       [ 3.39090454,  3.81110331],
       [ 0.65917771,  4.22063541],
       [-1.5931153 ,  4.14236792]])

##Multiple linear regression##


In [ ]:
import numpy as np

def predict(X, w, b):
    return np.dot(X, w) + b

def mean_absolute_error(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))

def gradient_descent(X, y, learning_rate=0.05, num_iterations=1000, tol=1e-6):
    m, n = X.shape
    w = np.zeros(n)
    b = 0
    cost_history = []

    for i in range(num_iterations):
        # Predict current prices
        y_pred = predict(X, w, b)

        # Calculate the gradient
        dw = -(2/m) * np.dot(X.T, (y - y_pred))
        db = -(2/m) * np.sum(y - y_pred)

        # Update the parameters
        w -= learning_rate * dw
        b -= learning_rate * db

        # Calculate and record the cost
        cost = mean_absolute_error(y, y_pred)
        cost_history.append(cost)

        # Check for convergence
        if len(cost_history) > 1 and np.abs(cost_history[-1] - cost_history[-2]) < tol:
            break

    return w, b, cost_history


w, b, cost_history = gradient_descent(xtrain, ytrain)

# To make predictions on the test set:
y_pred_test = predict(xtest, w, b)
y_pred_test


array([-0.43296948, -0.49976552,  0.00696488, ..., -0.66527864,
       -2.13474735,  0.48303296])

In [ ]:
import pandas as pd
import numpy as np

# Your NumPy array


# Assuming df_test is your DataFrame and it has a 'row_id' column
if 'row_id' in df_test.columns:
    # Check if the length of the array matches the number of rows in df_test
    if len(y_pred_test) == len(df_test):
        df = pd.DataFrame({
            'time_id': df_test['time_id'].values,
             'row_id': df_test['row_id'].values, # Use .values to ensure compatibility
            'target': y_pred_test
        })
    else:
        print("The length of the array does not match the number of rows in df_test.")
else:
    print("The 'row_id' column is not in df_test.")

print(df)

       time_id       row_id    target
0        26290      478_0_0 -0.302170
1        26290      478_0_1  0.949322
2        26290      478_0_2  1.530730
3        26290      478_0_3 -0.429660
4        26290      478_0_4 -0.369826
...        ...          ...       ...
32995    26454  480_540_195  0.147048
32996    26454  480_540_196 -0.706800
32997    26454  480_540_197  1.085113
32998    26454  480_540_198  1.004341
32999    26454  480_540_199 -0.778610

[33000 rows x 3 columns]


In [ ]:
df.to_csv('submission_wIK.csv' , index=False)

##Performance Evaluation##

In [ ]:
mae = mean_absolute_error(ytest, y_pred_test)
mae

6.346516305973633